In [30]:
from ArV_NeuroSynth import ArV_NeuroSynth
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
import scipy.io
import numpy as np # Import numpy with the alias np

mat = scipy.io.loadmat("A01T.mat")
data = mat['data']

for i in range(9):
    item = data[0, i]
    if isinstance(item, np.ndarray) and item.shape == (1, 1):
        value = item[0, 0]
    else:
        value = item
    print(f"Entry {i}: type={type(value)}, shape={getattr(value, 'shape', None)}")

# Corrected indentation for the second for loop
for i in range(9):
    struct = mat['data'][0, i][0, 0]
    print(f"Entry {i} field names:", struct.dtype.names)

Entry 0: type=<class 'numpy.void'>, shape=()
Entry 1: type=<class 'numpy.void'>, shape=()
Entry 2: type=<class 'numpy.void'>, shape=()
Entry 3: type=<class 'numpy.void'>, shape=()
Entry 4: type=<class 'numpy.void'>, shape=()
Entry 5: type=<class 'numpy.void'>, shape=()
Entry 6: type=<class 'numpy.void'>, shape=()
Entry 7: type=<class 'numpy.void'>, shape=()
Entry 8: type=<class 'numpy.void'>, shape=()
Entry 0 field names: ('X', 'trial', 'y', 'fs', 'classes', 'artifacts', 'gender', 'age')
Entry 1 field names: ('X', 'trial', 'y', 'fs', 'classes', 'artifacts', 'gender', 'age')
Entry 2 field names: ('X', 'trial', 'y', 'fs', 'classes', 'artifacts', 'gender', 'age')
Entry 3 field names: ('X', 'trial', 'y', 'fs', 'classes', 'artifacts', 'gender', 'age')
Entry 4 field names: ('X', 'trial', 'y', 'fs', 'classes', 'artifacts', 'gender', 'age')
Entry 5 field names: ('X', 'trial', 'y', 'fs', 'classes', 'artifacts', 'gender', 'age')
Entry 6 field names: ('X', 'trial', 'y', 'fs', 'classes', 'artifact

In [32]:
import scipy.io

mat = scipy.io.loadmat("A01T.mat")
data = mat['data']

# Print all field names and their shapes for the first struct
struct = data[0, 0][0, 0]
print("Field names:", struct.dtype.names)
for name in struct.dtype.names:
    arr = struct[name]
    print(f"Field '{name}': type={type(arr)}, shape={getattr(arr, 'shape', None)}")

Field names: ('X', 'trial', 'y', 'fs', 'classes', 'artifacts', 'gender', 'age')
Field 'X': type=<class 'numpy.ndarray'>, shape=(29683, 25)
Field 'trial': type=<class 'numpy.ndarray'>, shape=(0, 1)
Field 'y': type=<class 'numpy.ndarray'>, shape=(0, 1)
Field 'fs': type=<class 'numpy.ndarray'>, shape=(1, 1)
Field 'classes': type=<class 'numpy.ndarray'>, shape=(1, 4)
Field 'artifacts': type=<class 'numpy.ndarray'>, shape=(0, 1)
Field 'gender': type=<class 'numpy.ndarray'>, shape=(1,)
Field 'age': type=<class 'numpy.ndarray'>, shape=(1, 1)


In [33]:
from bci_mat_loader import load_bci_mat

dataloader, eeg_tensor = load_bci_mat("A01T.mat")

for x_batch, in dataloader:
    print(x_batch.shape)  # Should be [batch_size, 4, 256]
    break






ValueError: Could not locate EEG data structure. Check the .mat file keys manually.

In [ ]:
model = ArV_NeuroSynth(input_channels=4, seq_length=256, latent_dim=32, noise_dim=50).to(device)


In [ ]:
from ArV_NeuroSynth import compute_gradient_penalty
import torch.nn as nn

opt_vae = torch.optim.Adam(list(model.encoder.parameters()) + list(model.decoder.parameters()), lr=1e-4)
opt_g = torch.optim.Adam(model.G_latent.parameters(), lr=1e-4, betas=(0.5, 0.9))
opt_d = torch.optim.Adam(model.D_latent.parameters(), lr=1e-4, betas=(0.5, 0.9))

loss_fn = nn.MSELoss()
λ_gp = 10
β = 1e-5
epochs = 5

for epoch in range(epochs):
    model.train()
    total_vae_loss, total_d_loss, total_g_loss = 0, 0, 0
    for x_batch, in dataloader:
        x_batch = x_batch.to(device)

        # VAE
        x_recon, mu, logvar, z_real = model(x_batch, mode='vae')
        kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
        vae_loss = loss_fn(x_recon, x_batch) + β * kl
        opt_vae.zero_grad(); vae_loss.backward(); opt_vae.step()
        total_vae_loss += vae_loss.item()

        # GAN - Discriminator
        for _ in range(5):
            z_fake = model(noise_for_g_latent=torch.randn(x_batch.size(0), model.noise_dim).to(device), mode='generate_latent_from_noise')
            d_real = model(z_real, mode='discriminate_latent')
            d_fake = model(z_fake, mode='discriminate_latent')
            gp = compute_gradient_penalty(model.D_latent, z_real, z_fake)
            d_loss = d_fake.mean() - d_real.mean() + λ_gp * gp
            opt_d.zero_grad(); d_loss.backward(); opt_d.step()
            total_d_loss += d_loss.item()

        # GAN - Generator
        z_fake = model(noise_for_g_latent=torch.randn(x_batch.size(0), model.noise_dim).to(device), mode='generate_latent_from_noise')
        g_loss = -model(z_fake, mode='discriminate_latent').mean()
        opt_g.zero_grad(); g_loss.backward(); opt_g.step()
        total_g_loss += g_loss.item()

    print(f"Epoch {epoch+1} | VAE: {total_vae_loss:.4f} | D: {total_d_loss:.4f} | G: {total_g_loss:.4f}")


NameError: name 'dataloader' is not defined

In [ ]:
import matplotlib.pyplot as plt

model.eval()
with torch.no_grad():
    x_real = next(iter(dataloader))[0][:1].to(device)
    x_recon, _, _, _ = model(x_real, mode='vae')

plt.figure(figsize=(12, 4))
for ch in range(x_real.shape[1]):
    plt.plot(x_real.cpu()[0, ch], label=f"Real Ch{ch}")
    plt.plot(x_recon.cpu()[0, ch], '--', label=f"Reconstructed Ch{ch}")
plt.legend(); plt.title("Real vs Reconstructed EEG"); plt.grid(); plt.tight_layout(); plt.show()


NameError: name 'model' is not defined